In [20]:
import os
import glob
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [21]:
# curl https://natura.di.uminho.pt/~jj/pln/calao/dicionario.html > dicionario.html

with open("dicionario.html") as fp:
    soup = BeautifulSoup(fp)


In [22]:
entries = soup.findAll(class_= 'entry')    

dict_table = pd.DataFrame()

for entry in entries:
    #term
    entry_details = {}
    term = entry.find("div", attrs = {"class": "term"}).string
    entry_details["term"] = [term]
    
    #description
    information = entry.find("div", attrs = {"class": "inf"})
    if information is not None:
        description = information.find("dt")
        entry_details["description"] = [description and description.string]

        def add_if_exists(element, column):
            entry_details[column] = [
                element.find_next_sibling().string if element is not None
                else "none"] 
        
        #levels
        add_if_exists(information.find("dt", string="level:"), "level")

        #synonyms
        sinonyms_key = information.find("i", string="sin.:")
        if sinonyms_key:
            sinonyms_key = sinonyms_key.parent
        add_if_exists(sinonyms_key, "synonym")
    
    entry_details = pd.DataFrame.from_dict(entry_details)
    dict_table = pd.concat([dict_table, entry_details])

dict_table

,term,description,level,synonym
0,a altas horas da noite,muito tarde na noite,none,noite alta; noite cerrada; lá para as quinhentas
0,abafa-palhinha,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...
0,abafar,"esconder, esquecer, mudar de assunto, fazer co...",none,none
0,abafo,"festas, carinho",none,cafuné; carícia; meiguice; quitute; quindim; ...
0,abananado,"admirado, aturdido e sem palavras",coloquial,estupefacto; azuratado; aturdido; embasbacado...
...,...,...,...,...
0,zombi,"pessoa que, mesmo a pé, está sempre a dormir",none,dorminhoco; sonolento; cu de sono
0,zuate,"referente à região das nádegas ou ânus, freque...",calão,bunda; alcofa; cu; peida; rabo; bilha; trasei...
0,zuca,é um:,none,brazuca
0,zumba que zumba,None,none,none


In [23]:
pd.unique(dict_table.level)

array(['none', ' calão carroceiro', ' coloquial', ' calão', ' erudito',
       ' arcaico', ' calão muito carroceiro',
       ' calão estupidamente carroceiro', ' arcaico ( anos 80 )',
       ' arcaico ( medieval )', ' arcaico ( séc. XX )', ' infantil',
       ' arcaico ( anos 30 )', ' arcaico ( rural )',
       ' arcaico ( anos 70 )', ' arcaico ( anos 60 )',
       ' arcaico ( mas ainda bastante activo )',
       ' arcaico ( mitologia grega )'], dtype=object)

In [24]:
level_dict = {' calão carroceiro':3, ' coloquial':1, ' calão':2, ' calão muito carroceiro':4,' calão estupidamente carroceiro':5}


dict_table['scale'] = dict_table['level'].replace(level_dict)
dict_table['scale'] = dict_table['scale'].replace(['none', ' erudito',
       ' arcaico', ' arcaico ( anos 80 )',
       ' arcaico ( medieval )', ' arcaico ( séc. XX )', ' infantil',
       ' arcaico ( anos 30 )', ' arcaico ( rural )',
       ' arcaico ( anos 70 )', ' arcaico ( anos 60 )',
       ' arcaico ( mas ainda bastante activo )',
       ' arcaico ( mitologia grega )'],0)

dict_table.dtypes

term           object
description    object
level          object
synonym        object
scale           int64
dtype: object

In [25]:
dict_table['scale'].astype(int)

0    0
0    3
0    0
0    0
0    1
    ..
0    0
0    2
0    0
0    0
0    0
Name: scale, Length: 6528, dtype: int64

In [26]:
dict_table.reset_index(drop=True, inplace=True)
dict_table

,term,description,level,synonym,scale
0,a altas horas da noite,muito tarde na noite,none,noite alta; noite cerrada; lá para as quinhentas,0
1,abafa-palhinha,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...,3
2,abafar,"esconder, esquecer, mudar de assunto, fazer co...",none,none,0
3,abafo,"festas, carinho",none,cafuné; carícia; meiguice; quitute; quindim; ...,0
4,abananado,"admirado, aturdido e sem palavras",coloquial,estupefacto; azuratado; aturdido; embasbacado...,1
...,...,...,...,...,...
6523,zombi,"pessoa que, mesmo a pé, está sempre a dormir",none,dorminhoco; sonolento; cu de sono,0
6524,zuate,"referente à região das nádegas ou ânus, freque...",calão,bunda; alcofa; cu; peida; rabo; bilha; trasei...,2
6525,zuca,é um:,none,brazuca,0
6526,zumba que zumba,None,none,none,0


In [27]:
dict_table = dict_table[dict_table['scale'] > 0]
dict_table 

,term,description,level,synonym,scale
1,abafa-palhinha,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...,3
4,abananado,"admirado, aturdido e sem palavras",coloquial,estupefacto; azuratado; aturdido; embasbacado...,1
11,à beça,em grande quantidade,coloquial,None,1
13,abichanado,&bullet; homossexual masculino passivo,calão,arrombado; paneleiro; larilas; lelé; bicha; b...,2
20,abono de família,"órgão sexual masculino, pénis, pênis",coloquial,None,1
...,...,...,...,...,...
6508,xarro,None,coloquial,cavalo; heroína; charro,1
6511,xibungo,&bullet; homossexual masculino passivo,calão carroceiro,arrombado; paneleiro; larilas; lelé; bicha; b...,3
6512,xilindró,cadeia,coloquial,pildra; choça; prisão; prisa; calabouço; masm...,1
6518,xulo,é um:,calão carroceiro,none,3


In [28]:
dict_table.to_csv("dicionario_limpo.csv", index=False, sep="|")